In [ ]:
import cv2
import numpy as np
import os
import matplotlib.pyplot as plt

In [ ]:
calibration_images_filepaths = ["/usercode/images/topography/slice_20b.jpg", "/usercode/images/topography/slice_40b.jpg", "/usercode/images/topography/slice_60b.jpg"]
# Read the calibration images
# They represent the laser line on steps at 20 mm, 40 mm, and 60 mm.
calibration_imgs = [cv2.imread(filepath) for filepath in calibration_images_filepaths]

In [ ]:
# >>> Blur the red channel of each image, with a blurring kernel of size (5, 5)
blurred_slice_20_img = None
blurred_slice_40_img = None
blurred_slice_60_img = None

In [ ]:
# For each x, identify the y where the red intensity is maximum. Store y_max in three dictionaries.
# If the maximum intensity is below the threshold, store -1 instead.
minimum_red_value = 10
slice_img_sizeHW = blurred_slice_20_img.shape
x_to_ymax_20 = {}
x_to_ymax_40 = {}
x_to_ymax_60 = {}
for x in range(slice_img_sizeHW[1]):
    # >>> Extract a single column of each blurred image
    column_20_img = None
    column_40_img = None
    column_60_img = None
    # >>> Locate the y where the red intensity is maximum. 
    _, max_val_20, _, max_loc_20 = None
    _, max_val_40, _, max_loc_40 = None
    _, max_val_60, _, max_loc_60 = None
    if max_val_20 > minimum_red_value:
        x_to_ymax_20[x] = max_loc_20[1]  # We keep the y value
    else:
        x_to_ymax_20[x] = -1  # Flag to indicate that the signal is too weak
    if max_val_40 > minimum_red_value:
        x_to_ymax_40[x] = max_loc_40[1]  # We keep the y value
    else:
        x_to_ymax_40[x] = -1  # Flag to indicate that the signal is too weak
    if max_val_60 > minimum_red_value:
        x_to_ymax_60[x] = max_loc_60[1]  # We keep the y value
    else:
        x_to_ymax_60[x] = -1  # Flag to indicate that the signal is too weak

In [ ]:
# For each x, fit a line that best satisfies the three points (y_max, Y) where Y is the height in mm
x_to_a_b = {}
for x in range(slice_img_sizeHW[1]):
    if x_to_ymax_20[x] >= 0 and x_to_ymax_40[x] >= 0 and x_to_ymax_60[x] >= 0:
        p0 = (x_to_ymax_20[x], 20.0)
        p1 = (x_to_ymax_40[x], 40.0)
        p2 = (x_to_ymax_60[x], 60.0)
        # Solve an overdetermined system of linear equations:
        #      Az = d
        # | x_0   1 | | a |   | y_0 |
        # | x_1   1 | | b | = | y_1 |
        # | x_2   1 |         | y_2 |
        # >>> Build the A matrix and the d vector such that Az = d
        A = None
        d = None
        ab, residuals, rank, s = np.linalg.lstsq(A, d, rcond=None)
        a = ab[0, 0]
        b = ab[1, 0]
        x_to_a_b[x] = (a, b)
    else:
        x_to_a_b[x] = (None, None)


In [ ]:
# Plot the a, b parameters
x_list = list(range(slice_img_sizeHW[1]))
a_list = [x_to_a_b[x][0] for x in x_list]
b_list = [x_to_a_b[x][1] for x in x_list]
fig, axs = plt.subplots(2, 1)
axs[0].plot(x_list, a_list)
axs[0].set_xlim(0, slice_img_sizeHW[1] - 1)
axs[0].set_xlabel('x')
axs[0].set_ylabel('a')
axs[0].grid(True)
axs[1].plot(x_list, b_list)
axs[1].set_xlim(0, slice_img_sizeHW[1] - 1)
axs[1].set_xlabel('x')
axs[1].set_ylabel('b')
axs[1].grid(True)
fig.tight_layout()
plt.show()

In [ ]:
# Receive a slice image
slice_img = cv2.imread("/usercode/images/topography/grabbed_object_slice.jpg")
plt.imshow(cv2.cvtColor(slice_img, cv2.COLOR_BGR2RGB))


In [ ]:
# Convert the slice image into a topographic slice
# >>> Blur the red channel of slice_img
blurred_slice_img = None
heights = []
for x in range(blurred_slice_img.shape[1]):
    # >>> Locate the y where the intensity is maximum
    _, max_val, _, max_loc = None
    if max_val >= minimum_red_value:
        y = max_loc[1]  # (0, y_max)
        a = x_to_a_b[x][0]
        b = x_to_a_b[x][1]
        # >>> Reconstruct Y with a, b, and y
        heights.append(None)
    else:
        heights.append(-1)  # Special flag for 'unknown'

In [ ]:
# Display the topographic slice
xs = np.arange(0, len(heights))
fig, ax = plt.subplots()
ax.scatter(xs, heights, c="blue", marker='.')
ax.set_xlabel('x')
ax.set_ylabel('height (mm)')
plt.show()